In [1]:
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

set_nb_theme('chesterish')

In [2]:
#load libraries 
import time,os
from matplotlib.offsetbox import AnchoredText
import sys,pandas as pd, matplotlib , matplotlib.pyplot as plt, matplotlib.lines , numpy as np,cupy as cp, math, pylab
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
#import cudf
#import ROOT
#from ROOT import TFile

#import root_pandas

#from toggle_cell import hide_cell 
import uproot3
%matplotlib inline

#presets for plot style
from scipy.constants import golden
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (7, 7/golden),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

viridis = cm.get_cmap('viridis', 12)
inferno = cm.get_cmap('inferno', 12)
autumn = cm.get_cmap('autumn', 12)

#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)
rc = plt.rc
rc('text', usetex=False)
import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=False)
rc('font', size=15)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

plt.rcParams['figure.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
plt.rcParams['savefig.bbox']='tight'

import root_pandas,glob

import gc

def read_n(pattern, n):
    if not '*' in pattern:
        a = root_pandas.read_root(pattern,chunksize=int(n))
    else:
        a = root_pandas.read_root(glob.glob(pattern),chunksize=int(n))
    for b in a:
        del a
        print(n)
        return b

from scipy.optimize import curve_fit

image_dir='images'

import misc_util,correlation_plots

import imp
imp.reload(correlation_plots)

/home/sebouh/.local/lib/python3.8/site-packages/uproot3/__init__.py:127: FutureWarning: Consider switching from 'uproot3' to 'uproot', since the new interface became the default in 2020.

    pip install -U uproot

In Python:

    >>> import uproot
    >>> with uproot.open(...) as file:
    ...

  warnings.warn(
/home/sebouh/.local/lib/python3.8/site-packages/awkward0/__init__.py:12: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  warnings.warn(


<module 'correlation_plots' from '/home/sebouh/di-hadron/correlation_plots.py'>

In [5]:
#sets =  "D_C C D_Fe Fe Pb D_Pb".split()
#sets += "MC_D2 MC_C MC_Fe MC_Pb".split()
#sets += "no_ecuts_D_C no_ecuts_C no_ecuts_D_Fe no_ecuts_Fe no_ecuts_Pb no_ecuts_D_Pb".split()
#sets += "xQ2_D_C xQ2_C xQ2_D_Fe xQ2_Fe xQ2_Pb xQ2_D_Pb".split()

#dfs_mixed = {a : root_pandas.read_root("../../eg2/python/mixed_%s.root"%a)  for a in sets}

mixtag='xQ2_'
dfs_mixed = {a : root_pandas.read_root(f"/home/sebouh/eg2/python/mixed_{mixtag}{a}.root")  for a in "D_C C D_Fe Fe Pb D_Pb".split()}

In [6]:
for dct in dfs_mixed,:
    dct['D'] = pd.concat([dct['D_'+a] for a in 'C Fe Pb'.split()])
    print(dct.keys())

dict_keys(['D_C', 'C', 'D_Fe', 'Fe', 'Pb', 'D_Pb', 'D'])


In [7]:
for df in dfs_mixed.values():
    if 'h1_FidCutPiPlus' not in df.columns:
        df['h1_FidCutPiPlus'] = [1]*len(df)
        df['h2_FidCutPiPlus'] = [1]*len(df)

In [8]:
import R2h_module
import imp
imp.reload(R2h_module)
for key in list(dfs_mixed.keys()):
    if '_2pi' in key or '_pi_p' in key:
        continue
    print(f"\n\n{key}")
    if 'dphi' not in dfs_mixed[key].columns:
        dfs_mixed[key].eval('diff_phi_cm = h1_cm_ph-h2_cm_ph', inplace=True)
        dfs_mixed[key].eval(f'diff_phi_cm = diff_phi_cm-{2*np.pi}*(diff_phi_cm>{np.pi})', inplace=True)
        dfs_mixed[key].eval(f'diff_phi_cm = diff_phi_cm+{2*np.pi}*(diff_phi_cm<{-np.pi})', inplace=True)
        dfs_mixed[key].eval(f'dphi = abs(diff_phi_cm)', inplace=True)
    mixed_trigger_cuts = R2h_module.applyCuts(dfs_mixed[key])
    
    print(mixed_trigger_cuts)
        
    dfs_mixed[key + '_2pi'] = R2h_module.applyCutsPair(mixed_trigger_cuts,isMC=False,
                                                             nomCuts=True,h2Proton=False)
    dfs_mixed[key + '_pi_p'] = R2h_module.applyCutsPair(mixed_trigger_cuts,isMC=False,
                                                             nomCuts=True,h2Proton=True)
                                 
    for df in (dfs_mixed[key + '_2pi'],dfs_mixed[key + '_pi_p']):
        df.eval('diff_rap_cm=0.5*log((h1_e+sqrt(h1_p**2-h1_cm_pt**2))/(h1_e-sqrt(h1_p**2-h1_cm_pt**2)))-0.5*log((h2_e+sqrt(h2_p**2-h2_cm_pt**2))/(h2_e-sqrt(h2_p**2-h2_cm_pt**2)))',inplace=True)
        df.eval('diff_phi_cm = h1_cm_ph-h2_cm_ph', inplace=True)
        df.eval(f'diff_phi_cm = diff_phi_cm-{2*np.pi}*(diff_phi_cm>{np.pi})', inplace=True)
        df.eval(f'diff_phi_cm = diff_phi_cm+{2*np.pi}*(diff_phi_cm<{-np.pi})', inplace=True)



D_C
Entries before cut  3078859
Q2>1.0 and Q2<4.0 3078859  (100.00  %)
0.2<h1_p<5.0  3078859  (100.00  %)
inelasticity < 0.85 3078859  (100.00  %)
h1_pid = pions (trigger) 3078859  (100.00  %)
2.2 < nu <4.2 3037189  (98.65  %)
10< h1_th<90 3037189  (100.00  %)
Theta/P fiducial region selected for trigger 3037189  (100.00  %)
             E  mixevt_E       e_p  mixevt_e_p      e_th  mixevt_e_th  \
entry                                                                   
0        5.014     5.014  1.712664    2.069132  0.448546     0.376894   
1        5.014     5.014  1.712664    1.926008  0.448546     0.402084   
2        5.014     5.014  1.712664    1.525837  0.448546     0.492809   
3        5.014     5.014  1.712664    1.664888  0.448546     0.449163   
4        5.014     5.014  1.712664    1.664888  0.448546     0.449163   
...        ...       ...       ...         ...       ...          ...   
3078854  5.014     5.014  2.057893    1.712610  0.363822     0.386087   
3078855  5.014

1.0< Q2 <4.0 2246401  (100.00  %)
2.2 < nu < 4.2 2246401  (100.00  %)
secondary hadrons are pions 1417300  (63.09  %)
(h1_z+h2_z)<1.0 1258397  (88.79  %)
(h1_pid*h2_pid)<0, opposite charge pions 488749  (38.84  %)
kinematic limits for h2_z 486568  (99.55  %)
kinematic limits for dphi 486327  (99.95  %)
kinematic limits for pair_mass 482279  (99.17  %)
leading hadrons are pions 482279  (100.00  %)
0.2<h2_p<5.0 482279  (100.00  %)
0.2<h1_p<5.0 482279  (100.00  %)
10<h2_th<90 480788  (99.69  %)
Theta/P fiducial region selected for secondary hadron 480788  (100.00  %)
10< h1_th<90 480788  (100.00  %)
Theta/P fiducial region selected for trigger hadron 480788  (100.00  %)
Nom cuts for the pair applied (pi pi) 216003  (44.93  %)
Starting election on dipion variables
This is Data
1.0< Q2 <4.0 2246401  (100.00  %)
2.2 < nu < 4.2 2246401  (100.00  %)
secondary hadrons are protons 829101  (36.91  %)
leading hadrons are pions 829101  (100.00  %)
0.2<h2_p<5.0 829101  (100.00  %)
0.2<h1_p<5.0 82910

1.0< Q2 <4.0 1780423  (100.00  %)
2.2 < nu < 4.2 1780423  (100.00  %)
secondary hadrons are pions 931101  (52.30  %)
(h1_z+h2_z)<1.0 839217  (90.13  %)
(h1_pid*h2_pid)<0, opposite charge pions 332160  (39.58  %)
kinematic limits for h2_z 330867  (99.61  %)
kinematic limits for dphi 330696  (99.95  %)
kinematic limits for pair_mass 327411  (99.01  %)
leading hadrons are pions 327411  (100.00  %)
0.2<h2_p<5.0 327411  (100.00  %)
0.2<h1_p<5.0 327411  (100.00  %)
10<h2_th<90 325469  (99.41  %)
Theta/P fiducial region selected for secondary hadron 325469  (100.00  %)
10< h1_th<90 325469  (100.00  %)
Theta/P fiducial region selected for trigger hadron 325469  (100.00  %)
Nom cuts for the pair applied (pi pi) 145265  (44.63  %)
Starting election on dipion variables
This is Data
1.0< Q2 <4.0 1780423  (100.00  %)
2.2 < nu < 4.2 1780423  (100.00  %)
secondary hadrons are protons 849322  (47.70  %)
leading hadrons are pions 849322  (100.00  %)
0.2<h2_p<5.0 849322  (100.00  %)
0.2<h1_p<5.0 849322 

1.0< Q2 <4.0 4086460  (100.00  %)
2.2 < nu < 4.2 4086460  (100.00  %)
secondary hadrons are pions 2952348  (72.25  %)
(h1_z+h2_z)<1.0 2550662  (86.39  %)
(h1_pid*h2_pid)<0, opposite charge pions 987627  (38.72  %)
kinematic limits for h2_z 982322  (99.46  %)
kinematic limits for dphi 981852  (99.95  %)
kinematic limits for pair_mass 975869  (99.39  %)
leading hadrons are pions 975869  (100.00  %)
0.2<h2_p<5.0 975869  (100.00  %)
0.2<h1_p<5.0 975869  (100.00  %)
10<h2_th<90 974288  (99.84  %)
Theta/P fiducial region selected for secondary hadron 974288  (100.00  %)
10< h1_th<90 974288  (100.00  %)
Theta/P fiducial region selected for trigger hadron 974288  (100.00  %)
Nom cuts for the pair applied (pi pi) 440789  (45.24  %)
Starting election on dipion variables
This is Data
1.0< Q2 <4.0 4086460  (100.00  %)
2.2 < nu < 4.2 4086460  (100.00  %)
secondary hadrons are protons 1134112  (27.75  %)
leading hadrons are pions 1134112  (100.00  %)
0.2<h2_p<5.0 1134112  (100.00  %)
0.2<h1_p<5.0 11

In [9]:
#binning info
yrange_dfdy =(0,3.5)
bins_dfdy = (7,8)
#yrange_dfdy =(-0,3.5)
#bins_dfdy = (7,8)

In [24]:
#check if the M_A/M_D deviates from unity (conclusion:  it does deviate)
for topo in "_2pi _pi_p".split():
    print(topo)
    for key in "C Fe Pb".split():
        df1 = dfs_mixed[key+topo]
        df2 = dfs_mixed["D_"+key+topo]
        h1,xedges,yedges = np.histogram2d(df1.diff_rap_cm, np.abs(df1.diff_phi_cm), 
                       bins=(bins_dfdy[0],bins_dfdy[1]//2),range=((0,np.pi),yrange_dfdy))
        h2,xedges,yedges = np.histogram2d(df2.diff_rap_cm, np.abs(df2.diff_phi_cm), 
                       bins=(bins_dfdy[0],bins_dfdy[1]//2),range=((0,np.pi),yrange_dfdy))
        h3 = np.divide(h1,h2)*len(df2)/len(df1)
        
        dh3 = h3*np.sqrt(np.divide(1,h1)+np.divide(1,h2))
        print(h3)
        print(dh3)

_2pi
[[1.00992502 0.92333634 0.96914272 0.94473583]
 [0.91492103 0.94107827 0.95378986 0.95715288]
 [0.96493048 0.9588813  0.97402409 0.99502349]
 [1.0056236  1.01667264 1.05613952 1.06390853]
 [1.01082318 1.04176998 1.11893564 1.06836069]
 [1.04689881 1.08671667 1.1159946  1.0756641 ]
 [1.15005633 1.04804992 1.14713641 1.05067234]]
[[0.0188395  0.01670233 0.017748   0.02125802]
 [0.01113637 0.01153935 0.01206209 0.01516434]
 [0.00968633 0.00978181 0.01049455 0.01421669]
 [0.01043961 0.01097839 0.01232043 0.01686965]
 [0.01324502 0.01444699 0.01714277 0.02266757]
 [0.01944542 0.02191333 0.02506546 0.03334578]
 [0.03617689 0.03536601 0.04196241 0.05316381]]
[[1.0059303  0.94978261 0.96747395 0.96921041]
 [0.912815   0.93731288 0.96041093 0.94990047]
 [0.9336019  0.95413623 0.97708493 1.01666028]
 [0.99421643 1.02005087 1.06978218 1.06292229]
 [1.02573173 1.01247863 1.09387135 1.10668908]
 [1.05921415 1.07908592 1.06627209 1.09719709]
 [1.19068099 1.03574498 1.18124574 1.08433701]]
[[0.0

In [49]:
#check if the mixed event yield is the same between the three deuterium sets.  
#(spoiler, it does not)
from scipy.stats import chi2 as chi2lib
chi2sf = chi2lib.sf
from scipy.stats import norm as gauslib
gaussf = gauslib.sf

largest_z = 0
chi2tot = 0
for topo in "_2pi _pi_p".split():
    print(topo)
    for key in "C Fe Pb".split():
        df1 = dfs_mixed["D_"+key+topo]
        df2 = dfs_mixed['D'+topo]
        h1,xedges,yedges = np.histogram2d(df1.diff_rap_cm, np.abs(df1.diff_phi_cm), 
                       bins=(bins_dfdy[0],bins_dfdy[1]//2),range=((0,np.pi),yrange_dfdy))
        h2,xedges,yedges = np.histogram2d(df2.diff_rap_cm, np.abs(df2.diff_phi_cm), 
                       bins=(bins_dfdy[0],bins_dfdy[1]//2),range=((0,np.pi),yrange_dfdy))
        h3 = np.divide(h1,h2)*len(df2)/len(df1)
        
        dh3 = h3*np.sqrt(np.divide(1,h1)+np.divide(1,h2))
        print(h3)
        print(dh3)
        chi2 = sum(sum((h3-1)**2/dh3**2))
        pval = chi2sf(chi2, len(h3)*len(h3[0]))
        print(pval)
        chi2tot+= chi2
        largest_z = max(np.max(abs(h3-1)/dh3),largest_z)
pval = chi2sf(chi2tot, 6*len(h3)*len(h3[0]))
print(pval)
print(largest_z)
print(1-(1-gaussf(largest_z))**(6*28))
print

_2pi
[[0.99001224 0.99783467 0.99706354 1.00463061]
 [1.00373657 0.9962735  0.99856498 0.99922077]
 [1.0032667  1.01526446 1.00734392 0.99997159]
 [1.01121956 1.00232484 0.99526572 0.997516  ]
 [1.00040273 1.00784674 0.98549664 0.99242612]
 [0.98712215 0.96600635 0.97137916 0.97417475]
 [0.96665843 1.01893055 0.9882114  1.01252331]]
[[0.01324255 0.01260291 0.01294033 0.01590911]
 [0.00851189 0.00857872 0.00890739 0.01117922]
 [0.00713296 0.00733131 0.00771282 0.01021053]
 [0.00753605 0.00778758 0.00844237 0.01152912]
 [0.00941322 0.01013634 0.01115332 0.0153582 ]
 [0.01328509 0.01423559 0.01607818 0.02202484]
 [0.02259114 0.02501254 0.02690379 0.03727061]]
0.5707687678040272
[[1.00339511 1.00306704 0.98755554 0.97059535]
 [1.00526597 0.99541398 0.99929925 0.98451829]
 [1.00321154 0.99848687 0.99885054 0.99210749]
 [1.00145286 0.9929936  1.00056776 0.99945987]
 [0.99979852 1.00555587 1.00840138 1.00492428]
 [1.0086178  1.00925803 1.02608655 0.98446208]
 [1.00670284 1.03002396 1.03145974

<function print>